## Predictive Analytics Project - Santander Customer Transaction Prediction

## Basic Predictive Models

In [10]:
### All imports
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import L1L2
from lightgbm import LGBMClassifier as lgb
# import lightgbm as lgb

from skopt.space import Real, Categorical, Integer
from skopt import BayesSearchCV

from imblearn.over_sampling import SMOTE

from collections import Counter

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
### Load file and explore
train_data = pd.read_csv("D:\\Academics\\UMN-MSBA\\Term 2\\MSBA 6420 - Predictive Analytics\\Project\\train.csv")

In [3]:
test_data = pd.read_csv("D:\\Academics\\UMN-MSBA\\Term 2\\MSBA 6420 - Predictive Analytics\\Project\\test.csv")

In [4]:
df_train = train_data.drop(['ID_code',	'target'], 1)
df_test = test_data.drop(['ID_code'], 1)

In [5]:
### Data Seperation
X = train_data.drop(['ID_code',	'target'], 1)
y = train_data['target']

In [6]:
features = list(X.columns)

#### Feature creation from EDA

In [7]:
# Real/Synthetic data
# test = df_test.drop(['ID_code'], axis=1).values
test = df_test
unique_count = np.zeros_like(test)
for feature in range(test.shape[1]):
    _, index, count = np.unique(test.iloc[:, feature], return_counts=True, return_index=True)
    unique_count[index[count == 1], feature] += 1
real_samples = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synth_samples = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]
print('Number of real samples in test set is {}'.format(len(real_samples)))
print('Number of synthetic samples in test set is {}'.format(len(synth_samples)))

features = [col for col in df_train.columns if col.startswith('var')]
df_all = pd.concat([df_train, df_test.iloc[real_samples]])
for feature in features:
    temp = df_all[feature].value_counts(dropna=True)
    df_train[feature + 'vc'] = df_train[feature].map(temp).map(lambda x: min(10, x)).astype(np.uint8)
    df_test[feature + 'vc'] = df_test[feature].map(temp).map(lambda x: min(10, x)).astype(np.uint8)
    df_train[feature + 'sum'] = ((df_train[feature] - df_all[feature].mean()) * df_train[feature + 'vc'] \
                                 .map(lambda x: int(x > 1))).astype(np.float32)
    df_test[feature + 'sum'] = ((df_test[feature] - df_all[feature].mean()) * df_test[feature + 'vc'] \
                                .map(lambda x: int(x > 1))).astype(np.float32)
    df_train[feature + 'sum2'] = ((df_train[feature]) * df_train[feature + 'vc'] \
                                  .map(lambda x: int(x > 2))).astype(np.float32)
    df_test[feature + 'sum2'] = ((df_test[feature]) * df_test[feature + 'vc'] \
                                 .map(lambda x: int(x > 2))).astype(np.float32)
    df_train[feature + 'sum3'] = ((df_train[feature]) * df_train[feature + 'vc'] \
                                  .map(lambda x: int(x > 4))).astype(np.float32)
    df_test[feature + 'sum3'] = ((df_test[feature]) * df_test[feature + 'vc'] \
                                 .map(lambda x: int(x > 4))).astype(np.float32)
print('Training set shape after creating magic features: {}'.format(df_train.shape))
print('Test set shape after creating magic features: {}'.format(df_test.shape))

Number of real samples in test set is 100000
Number of synthetic samples in test set is 100000
Training set shape after creating magic features: (200000, 1000)
Test set shape after creating magic features: (200000, 1000)


### Train Test Split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size = 0.1, random_state = 42)

### Logistic Regression

In [10]:
## Pipeline creation for scaling
lr = LogisticRegression()
pipe_lr = Pipeline([('sc', StandardScaler()), ('lr', lr)])

In [11]:
## Bayes search for best parameters
bayes_cv_tuner_lr = BayesSearchCV(estimator = pipe_lr,
                                    search_spaces = {
                                                        'lr__class_weight': ['balanced'],
                                                        'lr__C': np.logspace(-5, 5, 11),
                                                         'lr__l1_ratio': np.logspace(-5, 0, 6),
                                                         'lr__penalty': ['elasticnet'],
                                                         'lr__solver': ['saga']
                                                    },
                                    scoring = 'roc_auc',
                                    cv = StratifiedKFold(
                                        n_splits=3,
                                        shuffle=True,
                                        random_state=42
                                    ),
                                    n_jobs = 3,
                                    n_iter = 10,
                                    verbose = 4,
                                    refit = True,
                                    random_state = 42
                                )

In [12]:
bayes_cv_tuner_lr.fit(X = X_train, y = y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
              estimator=Pipeline(steps=[('sc', StandardScaler()),
                                        ('lr', LogisticRegression())]),
              n_iter=10, n_jobs=3, random_state=42, scoring='roc_auc',
              search_spaces={'lr__C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                             'lr__class_weight': ['balanced'],
                             'lr__l1_ratio': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]),
                             'lr__penalty': ['elasticnet'],
                             'lr__solver': ['saga']},
              verbose=4)

In [14]:
bayes_cv_tuner_lr.best_params_

OrderedDict([('lr__C', 0.1),
             ('lr__class_weight', 'balanced'),
             ('lr__l1_ratio', 1.0),
             ('lr__penalty', 'elasticnet'),
             ('lr__solver', 'saga')])

In [15]:
y_pred_test = bayes_cv_tuner_lr.predict_proba(X = X_test)

In [17]:
roc_auc_score(y_test, y_pred_test[:,1])

0.8803438100747146

### Decision Tree

In [18]:
## Bayes search for best parameters
bayes_cv_tuner_dt = BayesSearchCV(estimator = DecisionTreeClassifier(),
                                    search_spaces = {
                                                        'class_weight': ['balanced'],
                                                        'criterion': ['gini'],
                                                        'max_depth': [1,2,3,4,5,6,7,8,9,10],
                                                        'max_leaf_nodes': [2,3,4,5,6,7,8,9,10],
                                                        'min_samples_leaf': [2,3,4,5,6,7,8,9,10],
                                                        'min_samples_split': [2,3,4,5,6,7,8,9,10]
                                                    },
                                    scoring = 'roc_auc',
                                    cv = StratifiedKFold(
                                        n_splits=3,
                                        shuffle=True,
                                        random_state=42
                                    ),
                                    n_jobs = 3,
                                    n_iter = 15,
                                    verbose = 100,
                                    refit = True,
                                    random_state = 42
                                )

In [19]:
bayes_cv_tuner_dt.fit(X = X_train, y = y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
              estimator=DecisionTreeClassifier(), n_iter=15, n_jobs=3,
              random_state=42, scoring='roc_auc',
              search_spaces={'class_weight': ['balanced'],
                             'criterion': ['gini'],
                             'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
              verbose=100)

In [20]:
bayes_cv_tuner_dt.best_params_

OrderedDict([('class_weight', 'balanced'),
             ('criterion', 'gini'),
             ('max_depth', 5),
             ('max_leaf_nodes', 9),
             ('min_samples_leaf', 5),
             ('min_samples_split', 2)])

In [25]:
dt = DecisionTreeClassifier(criterion  = 'gini',
                            class_weight = 'balanced',
                         max_depth = 5,
                         max_leaf_nodes = 9,
                         min_samples_leaf = 5,
                         min_samples_split = 2)

In [26]:
dt.fit(X = X_train, y = y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5, max_leaf_nodes=9,
                       min_samples_leaf=5)

In [27]:
y_final_pred = dt.predict_proba(X_test)

In [28]:
roc_auc_score(y_test, y_final_pred[:, 1])

0.6338731462947962

## NN

In [29]:
## Best Parameters taken from Neural Network File
def build_model():
    model = keras.Sequential()
    model.add(Dense(288, activation = 'relu', input_dim = 1000))
    # model.add(Dropout(0.2))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    auc = AUC()
    model.compile(
        optimizer='adam', #learning_rate = 0.0003069273045576997),
        loss = "binary_crossentropy",
        metrics = [auc]
    )
    return model

model2 = KerasClassifier(build_fn = build_model, verbose = 2, epochs = 5, shuffle = False)# , class_weight = {0:1, 1:10})
pipe_nn = Pipeline([('nn', model2)])

In [30]:
pipe_nn.fit(X = X_train, y = y_train, nn__epochs = 20, nn__class_weight = {0:1, 1:10}, nn__shuffle = False)

Epoch 1/20
5625/5625 - 14s - loss: 0.9902 - auc: 0.8264 - 14s/epoch - 2ms/step
Epoch 2/20
5625/5625 - 13s - loss: 0.8840 - auc: 0.8607 - 13s/epoch - 2ms/step
Epoch 3/20
5625/5625 - 14s - loss: 0.8694 - auc: 0.8657 - 14s/epoch - 2ms/step
Epoch 4/20
5625/5625 - 13s - loss: 0.8559 - auc: 0.8701 - 13s/epoch - 2ms/step
Epoch 5/20
5625/5625 - 14s - loss: 0.8477 - auc: 0.8727 - 14s/epoch - 2ms/step
Epoch 6/20
5625/5625 - 14s - loss: 0.8418 - auc: 0.8746 - 14s/epoch - 2ms/step
Epoch 7/20
5625/5625 - 14s - loss: 0.8358 - auc: 0.8765 - 14s/epoch - 2ms/step
Epoch 8/20
5625/5625 - 14s - loss: 0.8313 - auc: 0.8778 - 14s/epoch - 2ms/step
Epoch 9/20
5625/5625 - 13s - loss: 0.8271 - auc: 0.8791 - 13s/epoch - 2ms/step
Epoch 10/20
5625/5625 - 13s - loss: 0.8234 - auc: 0.8803 - 13s/epoch - 2ms/step
Epoch 11/20
5625/5625 - 14s - loss: 0.8220 - auc: 0.8807 - 14s/epoch - 2ms/step
Epoch 12/20
5625/5625 - 14s - loss: 0.8196 - auc: 0.8814 - 14s/epoch - 2ms/step
Epoch 13/20
5625/5625 - 14s - loss: 0.8166 - auc:

Pipeline(steps=[('nn',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000027D9C2FB9D0>)])

In [31]:
y_final_pred = pipe_nn.predict_proba(X_test)

In [32]:
roc_auc_score(y_test, y_final_pred[:,1])

0.8756306001347883

## Random Forest

In [80]:
## Bayes search for best parameters
bayes_cv_tuner_rf = BayesSearchCV(estimator = RandomForestClassifier(),
                                    search_spaces = {
                                                        'class_weight': ['balanced'],
                                                        'criterion': ['gini'],
                                                        'max_depth': [1,2,3,4,5,6,7,8,9,10],
                                                        'max_leaf_nodes': [2,3,4,5,6,7,8,9,10],
                                                        'min_samples_leaf': [2,3,4,5,6,7,8,9,10],
                                                        'min_samples_split': [2,3,4,5,6,7,8,9,10]
                                                    },
                                    scoring = 'roc_auc',
                                    cv = StratifiedKFold(
                                        n_splits=3,
                                        shuffle=True,
                                        random_state=42
                                    ),
                                    n_jobs = 3,
                                    n_iter = 15,
                                    verbose = 100,
                                    refit = True,
                                    random_state = 42
                                )

In [81]:
bayes_cv_tuner_rf.fit(X = X_train, y = y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


BayesSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
              estimator=RandomForestClassifier(), n_iter=15, n_jobs=3,
              random_state=42, scoring='roc_auc',
              search_spaces={'class_weight': ['balanced'],
                             'criterion': ['gini'],
                             'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                             'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
              verbose=100)

In [86]:
bayes_cv_tuner_rf.best_params_

OrderedDict([('class_weight', 'balanced'),
             ('criterion', 'gini'),
             ('max_depth', 6),
             ('max_leaf_nodes', 8),
             ('min_samples_leaf', 6),
             ('min_samples_split', 3)])

In [84]:
y_final_pred = bayes_cv_tuner_rf.predict_proba(X = X_test)

In [85]:
roc_auc_score(y_test, y_final_pred)

0.6938715045677362

## Naive Bayes Classifier

In [33]:
nb = GaussianNB()

In [34]:
nb.fit(X = X_train, y = y_train)

GaussianNB()

In [35]:
y_pred = nb.predict_proba(X = X_test)

In [36]:
roc_auc_score(y_test, y_pred[:, 1])

0.8044284938302064

## Stacking Classifier

In [11]:
## Best Logistic Regression Model
lr_best = LogisticRegression(C= 0.1,
                             class_weight= 'balanced',
                             l1_ratio= 1,
                             penalty= 'elasticnet',
                             solver= 'saga')
pipe_lr_best = Pipeline([('sc', StandardScaler()), ('lr', lr_best)])

## Naive Bayes
nb = GaussianNB()

## Best Decision Tree Model
dt = DecisionTreeClassifier(criterion  = 'gini',
                            class_weight = 'balanced',
                            max_depth = 5,
                            max_leaf_nodes = 9,
                            min_samples_leaf = 5,
                            min_samples_split = 2)

## Best Random Forest Model
rf = RandomForestClassifier(class_weight = 'balanced',
                            criterion = 'gini',
                            max_depth = 6,
                            max_leaf_nodes = 8,
                            min_samples_leaf = 6,
                            min_samples_split = 3
                           )

## Best parameters taken from LGBM file
lgbm = lgb(boost_from_average = True,
          colsample_bytree = 0.5178695836798571,
          is_unbalance = True,
          learning_rate = 0.1,
          max_depth = -1,
          metric = 'auc',
          min_data_in_leaf = 143,
          num_leaves = 151,
          objective = 'binary',
          subsample = 1,
          subsample_freq = 7          
         )

### Best Neural Network Model
def build_model():
    model = keras.Sequential()
    model.add(Dense(288, activation = 'relu', input_dim = 1000))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    auc = AUC()
    model.compile(
        optimizer='adam',
        loss = "binary_crossentropy",
        metrics = [auc]
    )
    return model

model2 = KerasClassifier(build_fn = build_model, verbose = 2, epochs = 8, shuffle = False, class_weight = {0:1, 1:10})
model2._estimator_type = 'classifier'
pipe_nn = Pipeline([('nn', model2)])

In [12]:
## Base estimators chosen as LR, NN, NB, RF and DT with final Estimator as LGBM
est = [('lr', pipe_lr_best), ('nn', pipe_nn), ('nb', nb), ('rf', rf), ('dt', dt)]
final_estimator = lgbm

In [13]:
## Stacking CLassifier with passthrough True as Passthrough False might not be enough for predictions
stack = StackingClassifier(estimators = est,
                           final_estimator = final_estimator,
                           passthrough = True,
                           stack_method = 'predict_proba',
                           verbose = 4,
                           n_jobs = 1
                          )

In [14]:
stack.fit(X = X_train, y = y_train)

Epoch 1/8
5625/5625 - 13s - loss: 0.9901 - auc: 0.8262 - 13s/epoch - 2ms/step
Epoch 2/8
5625/5625 - 12s - loss: 0.8856 - auc: 0.8600 - 12s/epoch - 2ms/step
Epoch 3/8
5625/5625 - 12s - loss: 0.8678 - auc: 0.8662 - 12s/epoch - 2ms/step
Epoch 4/8
5625/5625 - 12s - loss: 0.8554 - auc: 0.8703 - 12s/epoch - 2ms/step
Epoch 5/8
5625/5625 - 12s - loss: 0.8471 - auc: 0.8729 - 12s/epoch - 2ms/step
Epoch 6/8
5625/5625 - 12s - loss: 0.8417 - auc: 0.8746 - 12s/epoch - 2ms/step
Epoch 7/8
5625/5625 - 13s - loss: 0.8388 - auc: 0.8756 - 13s/epoch - 2ms/step
Epoch 8/8
5625/5625 - 13s - loss: 0.8341 - auc: 0.8771 - 13s/epoch - 2ms/step


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  7.3min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 12.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/8
4500/4500 - 10s - loss: 1.0721 - auc_1: 0.8147 - 10s/epoch - 2ms/step
Epoch 2/8
4500/4500 - 10s - loss: 0.9148 - auc_1: 0.8503 - 10s/epoch - 2ms/step
Epoch 3/8
4500/4500 - 10s - loss: 0.8820 - auc_1: 0.8618 - 10s/epoch - 2ms/step
Epoch 4/8
4500/4500 - 10s - loss: 0.8667 - auc_1: 0.8667 - 10s/epoch - 2ms/step
Epoch 5/8
4500/4500 - 10s - loss: 0.8593 - auc_1: 0.8690 - 10s/epoch - 2ms/step
Epoch 6/8
4500/4500 - 10s - loss: 0.8522 - auc_1: 0.8714 - 10s/epoch - 2ms/step
Epoch 7/8
4500/4500 - 10s - loss: 0.8479 - auc_1: 0.8727 - 10s/epoch - 2ms/step
Epoch 8/8
4500/4500 - 10s - loss: 0.8438 - auc_1: 0.8740 - 10s/epoch - 2ms/step


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.4min remaining:    0.0s


Epoch 1/8
4500/4500 - 10s - loss: 1.1580 - auc_2: 0.8102 - 10s/epoch - 2ms/step
Epoch 2/8
4500/4500 - 10s - loss: 0.9329 - auc_2: 0.8446 - 10s/epoch - 2ms/step
Epoch 3/8
4500/4500 - 10s - loss: 0.9257 - auc_2: 0.8464 - 10s/epoch - 2ms/step
Epoch 4/8
4500/4500 - 10s - loss: 0.8945 - auc_2: 0.8570 - 10s/epoch - 2ms/step
Epoch 5/8
4500/4500 - 10s - loss: 0.8818 - auc_2: 0.8615 - 10s/epoch - 2ms/step
Epoch 6/8
4500/4500 - 10s - loss: 0.8953 - auc_2: 0.8586 - 10s/epoch - 2ms/step
Epoch 7/8
4500/4500 - 10s - loss: 0.8679 - auc_2: 0.8665 - 10s/epoch - 2ms/step
Epoch 8/8
4500/4500 - 10s - loss: 0.8654 - auc_2: 0.8674 - 10s/epoch - 2ms/step


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.1min remaining:    0.0s


Epoch 1/8
4500/4500 - 11s - loss: 1.0147 - auc_4: 0.8181 - 11s/epoch - 2ms/step
Epoch 2/8
4500/4500 - 10s - loss: 0.8898 - auc_4: 0.8587 - 10s/epoch - 2ms/step
Epoch 3/8
4500/4500 - 10s - loss: 0.8695 - auc_4: 0.8656 - 10s/epoch - 2ms/step
Epoch 4/8
4500/4500 - 10s - loss: 0.8569 - auc_4: 0.8698 - 10s/epoch - 2ms/step
Epoch 5/8
4500/4500 - 10s - loss: 0.8473 - auc_4: 0.8729 - 10s/epoch - 2ms/step
Epoch 6/8
4500/4500 - 10s - loss: 0.8415 - auc_4: 0.8747 - 10s/epoch - 2ms/step
Epoch 7/8
4500/4500 - 10s - loss: 0.8354 - auc_4: 0.8766 - 10s/epoch - 2ms/step
Epoch 8/8
4500/4500 - 10s - loss: 0.8303 - auc_4: 0.8781 - 10s/epoch - 2ms/step
Epoch 1/8
4500/4500 - 12s - loss: 1.0188 - auc_5: 0.8153 - 12s/epoch - 3ms/step
Epoch 2/8
4500/4500 - 11s - loss: 0.8927 - auc_5: 0.8576 - 11s/epoch - 2ms/step
Epoch 3/8
4500/4500 - 11s - loss: 0.8758 - auc_5: 0.8634 - 11s/epoch - 2ms/step
Epoch 4/8
4500/4500 - 11s - loss: 0.8623 - auc_5: 0.8680 - 11s/epoch - 2ms/step
Epoch 5/8
4500/4500 - 10s - loss: 0.8533

[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  7.0min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   15.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  8.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.2s remain

[LightGBM] [Warning] min_data_in_leaf is set=143, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=143


StackingClassifier(estimators=[('lr',
                                Pipeline(steps=[('sc', StandardScaler()),
                                                ('lr',
                                                 LogisticRegression(C=0.1,
                                                                    class_weight='balanced',
                                                                    l1_ratio=1,
                                                                    penalty='elasticnet',
                                                                    solver='saga'))])),
                               ('nn',
                                Pipeline(steps=[('nn',
                                                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x0000021FD1D2C670>)])),
                               ('nb', GaussianNB()),
                               ('rf',
                                RandomForestClassifier(class_weight='...
                     

In [15]:
y_pred = stack.predict_proba(X = X_test)

In [16]:
roc_auc_score(y_test, y_pred[:, 1])

0.9090680840764225